In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import lightning.pytorch as pl
pl.seed_everything(0)

Seed set to 0


0

# Set up
*   Install libraries
*   Load dataset

In [ ]:
# ! rm -rf dataset
# ! rm -rf library

In [ ]:
! pip show terratorch

## Setting for Google Colab Notebook

If you run on Google Colab Notebook, install python library first by using command below.


In [ ]:
# ! pip install xarray-spatial git+https://github.com/IBM/terratorch.git@fix/506

In [ ]:
# import terratorch
# pip install granitewxc

In [ ]:
# ! mkdir -p library
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/__init__.py -P library
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/analysis.py -P library

In [ ]:
# ! mkdir -p library/datasets
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/datasets/__init__.py -P library/datasets
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/datasets/sen2cloud.py -P library/datasets

In [ ]:
# ! mkdir -p library/datamodules
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/datamodules/__init__.py -P library/datamodules
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/datamodules/sen2cloud.py -P library/datamodules

In [ ]:
# ! mkdir -p library/decoders
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/decoders/__init__.py -P library/decoders
# ! wget https://github.com/hk-kaden-kim/S2-CloudCover/raw/refs/heads/main/library/decoders/unetplusplus.py -P library/decoders

## Load Dataset

In [ ]:
# ! mkdir -p dataset

In [ ]:
# ! wget https://huggingface.co/datasets/hk-kaden-kim/Small_S2_CloudCover_Seg/resolve/main/train.zip -P dataset
# ! unzip -q dataset/train.zip -d dataset
# ! mv dataset/train dataset/public
# ! rm dataset/train.zip

In [ ]:
# ! wget https://huggingface.co/datasets/hk-kaden-kim/Small_S2_CloudCover_Seg/resolve/main/test.zip -P dataset
# ! unzip -q dataset/test.zip -d dataset
# ! mv dataset/test dataset/private
# ! rm dataset/test.zip

In [ ]:
# ! rm -rf dataset/__MACOSX

# Lightning Trainers

In [3]:
from library.datamodules.sen2cloud import Sen2CloudDataModule

BATCH_SIZE = 8 # 32

datamodule = Sen2CloudDataModule(
    data_root = './dataset',
    batch_size = BATCH_SIZE,
    means = [2672.63818359375, 2678.138671875, 2587.265380859375, 3507.404052734375],
    stds = [3047.300537109375, 2805.623779296875, 2705.935791015625, 2409.601318359375],
)

datamodule.setup("fit")
datamodule.setup("test")

train_dataset = datamodule.train_dataset
val_dataset = datamodule.val_dataset

test_dataset = datamodule.test_dataset
len(train_dataset), len(val_dataset), len(test_dataset)

(414, 171, 547)

In [5]:
from terratorch.tasks import SemanticSegmentationTask
from library.decoders.unetplusplus import UNetPlusPlusDecoder

LOSS = 'ce'
LEARNING_RATE = 1e-3
OPTIMIZER = 'AdamW'
OPTIMIZER_HPARAMS = {"weight_decay": 0.05}

# PRETRAINED, FREEZE = False, {'Encoder': False, 'Decoder': False} # Model 1
# PRETRAINED, FREEZE = True, {'Encoder': False, 'Decoder': False} # Model 2
PRETRAINED, FREEZE = True, {'Encoder': True, 'Decoder': False} # Model 3

model_args={
    # Backbone (Encoder)
    "backbone": "prithvi_eo_v2_300", # terratorch : prithvi_eo_v1_100, prithvi_eo_v2_300, prithvi_eo_v2_300_tl
    "backbone_kwargs": {
        'pretrained': PRETRAINED,
        'bands': ["BLUE", "GREEN", "RED", "NIR_NARROW"], # Resue NIR_NARROW (0.85µm~0.88µm) for NIR (0.78µm~0.88µm)
        'img_size':512,
    },

    # Necks
    "necks": [
        {
            "name": "SelectIndices",
            # "indices": [2, 5, 8, 11] # indices for prithvi_eo_v1_100
            "indices": [5, 11, 17, 23] # indices for prithvi_eo_v2_300
            # "indices": [7, 15, 23, 31] # indices for prithvi_eo_v2_600
        },
        {"name": "ReshapeTokensToImage",},
        {"name": "LearnedInterpolateToPyramidal"}
    ],

    # Decoder
    "decoder": "FCNDecoder",      # terratorch : FCNDecoder
    "decoder_kwargs": {
        'channels' : 256,
        'num_convs' : 4,
    },

    # Head
    "head_dropout": 0.1,
    "num_classes": 2,
}

# Model
task = SemanticSegmentationTask(
    model_args=model_args,
    model_factory="EncoderDecoderFactory",
    loss=LOSS,
    lr=LEARNING_RATE,
    optimizer=OPTIMIZER,
    optimizer_hparams=OPTIMIZER_HPARAMS,
    freeze_backbone=FREEZE['Encoder'], # True. Only to speed up fine-tuning
    freeze_decoder=FREEZE['Decoder'],
    class_names=['No', 'Cloud'],  # optionally define class names
    plot_on_val=0,
)

INFO:root:Loaded weights for HLSBands.BLUE in position 0 of patch embed
INFO:root:Loaded weights for HLSBands.GREEN in position 1 of patch embed
INFO:root:Loaded weights for HLSBands.RED in position 2 of patch embed
INFO:root:Loaded weights for HLSBands.NIR_NARROW in position 3 of patch embed


In [6]:
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint, RichProgressBar
from lightning.pytorch.loggers import TensorBoardLogger

EPOCH = 50

checkpoint_callback = ModelCheckpoint(mode="max",
                                      monitor="val/Multiclass_Jaccard_Index",
                                      filename="best-{epoch:02d}",
                                      save_last=True)
# early_stopping_callback = EarlyStopping(mode="min",
#                                         monitor=f"val/loss",
#                                         patience=10)
logger = TensorBoardLogger(save_dir='output',
                           version=
                           f"{'FT_' if PRETRAINED else ''}"\
                           f"{'Enc_' if not FREEZE['Encoder'] else ''}{'Dec_' if not FREEZE['Decoder'] else ''}"\
                           f"E{EPOCH}_B{BATCH_SIZE}_{LOSS}_LR{LEARNING_RATE}",
                           name=f"{model_args['backbone']}_{model_args['decoder']}")

trainer = Trainer(
    devices=1, # Number of GPUs. Interactive mode recommended with 1 device
    precision="16-mixed",
    callbacks=[
      RichProgressBar(),
      checkpoint_callback, # Set to save max val/IoU and last model
      # early_stopping_callback,
      LearningRateMonitor(logging_interval="epoch"),
      ],
    logger=logger,
    max_epochs=EPOCH,
    default_root_dir='output',
    log_every_n_steps=1,
    check_val_every_n_epoch=1,
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [7]:
import torch

torch.cuda.empty_cache()

try:
    free_mem, total_mem = torch.cuda.mem_get_info()
    print(f"Free Memory: {free_mem/1024**2} MB")
    print(f"Total Memory: {total_mem/1024**2} MB")
    print(f"Memory Allocated: {torch.cuda.memory_allocated(0)/1024**2} MB")
    print(f"Memory Reserved: {torch.cuda.memory_reserved(0)/1024**2} MB")
    print(f"Max Memory Allocated: {torch.cuda.max_memory_allocated(0)/1024**2} MB")
    print(f"Max Memory Reserved: {torch.cuda.max_memory_reserved(0)/1024**2} MB")
except Exception as e:
    print(e)


Free Memory: 5452.5 MB
Total Memory: 22502.8125 MB
Memory Allocated: 0.0 MB
Memory Reserved: 0.0 MB
Max Memory Allocated: 0.0 MB
Max Memory Reserved: 0.0 MB


In [ ]:
_ = trainer.fit(model=task, datamodule=datamodule)

In [ ]:
res = trainer.test(model=task, datamodule=datamodule) # Check default saved model here

In [ ]:
# ! zip -r prithvi_eo_v1_100_UNetPlusPlusDecoder.zip ./output/prithvi_eo_v1_100_UNetPlusPlusDecoder

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

# CLI tool

You find an example for SMP models in `configs/burnscars_smp.yaml` that you can run with `terratorch fit -c configs/burnscars_smp.yaml`.